In [23]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256
Corr>0.95: 139 Now:
Corr Target <0.1: 4206


## Test Improvement

In [28]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(np.sqrt(np.mean((y_pred - y_true)**2))) 

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

y_true,y_pred = y_test,clf.predict(x_test) 
print(root_mean_squared_error(y_pred, y_true))


1.4814606366304393


In [24]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

y_true,y_pred = y_test,clf.predict(x_test) 
print(root_mean_squared_error(y_pred, y_true))

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(clf.predict(x_train).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(clf.predict(x_test).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

Tensor("Sqrt_1:0", shape=(), dtype=float64)
Train on 3567 samples, validate on 892 samples
Epoch 1/1000
3567/3567 [==============================] - 0s 63us/step - loss: 16.5334 - val_loss: 9.3119
Epoch 2/1000
3567/3567 [==============================] - 0s 16us/step - loss: 9.2558 - val_loss: 6.5989
Epoch 3/1000
3567/3567 [==============================] - 0s 15us/step - loss: 8.3124 - val_loss: 6.1482
Epoch 4/1000
3567/3567 [==============================] - 0s 15us/step - loss: 7.5307 - val_loss: 6.1363
Epoch 5/1000
3567/3567 [==============================] - 0s 15us/step - loss: 6.9078 - val_loss: 4.4005
Epoch 6/1000
3567/3567 [==============================] - 0s 15us/step - loss: 6.1751 - val_loss: 4.5454
Epoch 7/1000
3567/3567 [==============================] - 0s 16us/step - loss: 5.6263 - val_loss: 5.2110
Epoch 8/1000
3567/3567 [==============================] - 0s 16us/step - loss: 5.1884 - val_loss: 5.2894
Epoch 9/1000
3567/3567 [==============================] - 0s 15us/st

Epoch 76/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.1875 - val_loss: 1.2727
Epoch 77/1000
3567/3567 [==============================] - 0s 17us/step - loss: 1.1838 - val_loss: 1.2726
Epoch 78/1000
3567/3567 [==============================] - 0s 17us/step - loss: 1.1864 - val_loss: 1.2724
Epoch 79/1000
3567/3567 [==============================] - 0s 17us/step - loss: 1.1793 - val_loss: 1.2724
Epoch 80/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.1801 - val_loss: 1.2723
Epoch 81/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.1861 - val_loss: 1.2719
Epoch 82/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.1897 - val_loss: 1.2718


## NN+ElNet - Actual

In [26]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)))

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(clf.predict(x_train).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(clf.predict(x_test).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s 64us/step - loss: 13.4618
Epoch 2/1000
4459/4459 [==============================] - 0s 14us/step - loss: 9.3323
Epoch 3/1000
4459/4459 [==============================] - 0s 14us/step - loss: 7.8107
Epoch 4/1000
4459/4459 [==============================] - 0s 14us/step - loss: 6.8561
Epoch 5/1000
4459/4459 [==============================] - 0s 14us/step - loss: 5.9459
Epoch 6/1000
4459/4459 [==============================] - 0s 14us/step - loss: 5.1325
Epoch 7/1000
4459/4459 [==============================] - 0s 14us/step - loss: 4.4513
Epoch 8/1000
4459/4459 [==============================] - 0s 14us/step - loss: 4.0552
Epoch 9/1000
4459/4459 [==============================] - 0s 14us/step - loss: 3.8638
Epoch 10/1000
4459/4459 [==============================] - 0s 14us/step - loss: 3.1641
Epoch 11/1000
4459/4459 [==============================] - 0s 15us/step - loss: 2.6941
Epoch 12/1000
4459/4459 [==========================

4459/4459 [==============================] - 0s 14us/step - loss: 1.1619
Epoch 96/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1775
Epoch 97/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1844
Epoch 98/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1720
Epoch 99/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1694
Epoch 100/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1708
Epoch 101/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1641
Epoch 102/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1667
Epoch 103/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1658
Epoch 104/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1643
Epoch 105/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1803
Epoch 106/1000
4459/4459 [=========================

4459/4459 [==============================] - 0s 15us/step - loss: 1.1057
Epoch 189/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1066
Epoch 190/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1040
Epoch 191/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0901
Epoch 192/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1016
Epoch 193/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1202
Epoch 194/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1078
Epoch 195/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1124
Epoch 196/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1051
Epoch 197/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1071
Epoch 198/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.1150
Epoch 199/1000
4459/4459 [=====================

In [27]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_boost.csv',index=False)

49342/49342 [==============================] - 1s 16us/step
          ID      target
0  000137c73  1752246.00
1  00021489f  2149595.00
2  0004d7953  2535024.25
3  00056a333  2129491.00
4  00056d8eb  2149595.00
